<a href="https://colab.research.google.com/github/petroslamb/my-things/blob/master/Alzheimer's_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Much of this notebook is based on work from Jay Alammar*

In [1]:
!pip install transformers

     |████████████████████████████████| 501kB 3.4MB/s 
     |████████████████████████████████| 870kB 62.4MB/s 
     |████████████████████████████████| 3.7MB 40.7MB/s 
     |████████████████████████████████| 1.0MB 42.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=6936e30b5d75a87419adf243f7b0c589a0b227a2ccb1663165a5f360fa092c61
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

## Importing the dataset
We'll use pandas to read the dataset and load it into a dataframe.

In [16]:
# Make data directory if it doesn't exist
!mkdir -p data
!unzip -n -d data ./AssignmentData.zip


Archive:  ./AssignmentData.zip


In [8]:
!ls ./data/AssignmentData/Assigment/AD/

01.txt	04.txt	07.txt	10.txt	13.txt	16.txt	19.txt	22.txt	25.txt	28.txt
02.txt	05.txt	08.txt	11.txt	14.txt	17.txt	20.txt	23.txt	26.txt	29.txt
03.txt	06.txt	09.txt	12.txt	15.txt	18.txt	21.txt	24.txt	27.txt	30.txt


Load data and create dataframe for those tested as AD positive.

In [58]:
import glob
import os

ad_path = './data/AssignmentData/Assigment/AD'
ad_filenames = glob.glob(os.path.join(ad_path, '*.txt'))
ad_data = []

for fn in ad_filenames:
  with open(fn, 'r') as f:
    ad_data.append(f.read().splitlines())

# See the form of the imported data
print(ad_data[0])
# Count the number of words per patient file
print([sum([len(s.split(' ')) for s in p]) for p in ad_data])

['oh boy .', 'alright .', 'family is in the kitchen .', "the mother's washing dishes .", 'and her sink is overflowing .', "and she's looking out the window .", "and the two kids are taking they're stealing cookies off the out_of the cupboard .", "and the boy looks like he's gonna  fall down and hurt himself or fall against his mother .", "and the girl is whispering “don't make too much noise” to him .", "she's &let or else she's laughin at him .", 'they got the cookies .', "alright now though the window, let's see .", "there's a nice look outside, real nice .", 'I told you the water was running over and splashing onto the floor .', "and the mother doesn't seem too too affected by it .", "she's dryin a dish or wiping it .", "let's see .", "I guess the girl is laughing at her brother because he's going to fall .", 'looks like a nice house .', "there is a little bit of very little  but I don't think that's meant for this .", 'the corner that got to the corner .', 'so this is a corner here

All the above passages are within the 512 Bert input limit. Which is good as we can use Bert easily.

In [67]:
import pandas as pd

ad_data_concat = [(" ").join(s) for s in ad_data]
print(ad_data_concat[0])

ad_df = pd.DataFrame(ad_data_concat, columns=['text'])
ad_df['label'] = pd.Series([1 for x in range(len(ad_df.index))], index=ad_df.index)
ad_df.head()

oh boy . alright . family is in the kitchen . the mother's washing dishes . and her sink is overflowing . and she's looking out the window . and the two kids are taking they're stealing cookies off the out_of the cupboard . and the boy looks like he's gonna  fall down and hurt himself or fall against his mother . and the girl is whispering “don't make too much noise” to him . she's &let or else she's laughin at him . they got the cookies . alright now though the window, let's see . there's a nice look outside, real nice . I told you the water was running over and splashing onto the floor . and the mother doesn't seem too too affected by it . she's dryin a dish or wiping it . let's see . I guess the girl is laughing at her brother because he's going to fall . looks like a nice house . there is a little bit of very little  but I don't think that's meant for this . the corner that got to the corner . so this is a corner here . and that goes back into there but that's  . do you see what I'

,text,label
0,oh boy . alright . family is in the kitchen . ...,1
1,what's happening there ? oh my . poor kids ...,1
2,oh little boy's in the cookie jar . the girl's...,1
3,there's a little girl reaching for the cookie...,1
4,here's a cookie jar . and the lid is off the c...,1


Let's do the same for the negative samples with label 0 this time.

In [60]:
import glob
import os

nc_path = './data/AssignmentData/Assigment/NC'
nc_filenames = glob.glob(os.path.join(nc_path, '*.txt'))
nc_data = []

for fn in nc_filenames:
  with open(fn, 'r') as f:
    nc_data.append(f.read().splitlines())

# See the form of the imported data
print(nc_data[0])
# Count the number of words per patient file
print([sum([len(s.split(' ')) for s in p]) for p in nc_data])

['a boy is getting cookies out of the cookie jar . ', "he's standing on a stool that's gonna fall . ", 'the girl is reaching for a cookie . ', "the mother's drying dishes . ", "the faucet's running water . ", "it's dripping out of the sink . ", 'spilling onto the floor . ', 'dishes are on the counter . ', 'window is open .', 'must be summertime . ', 'the girl is laughing . ', "looks like she's laughing . ", "that's about it . "]
[90, 120, 89, 303, 180, 165, 62, 140, 120, 78, 197, 185, 305, 199, 180, 198, 78, 111, 93, 70, 212, 221, 277, 122, 123, 124, 203, 146, 117, 131]


In [66]:
import pandas as pd

nc_data_concat = [(" ").join(s) for s in nc_data]
print(nc_data_concat[0])

nc_df = pd.DataFrame(nc_data_concat, columns=['text'])
nc_df['label'] = pd.Series([0 for x in range(len(nc_df.index))], index=nc_df.index)
nc_df.head()

a boy is getting cookies out of the cookie jar .  he's standing on a stool that's gonna fall .  the girl is reaching for a cookie .  the mother's drying dishes .  the faucet's running water .  it's dripping out of the sink .  spilling onto the floor .  dishes are on the counter .  window is open . must be summertime .  the girl is laughing .  looks like she's laughing .  that's about it . 


,text,label
0,a boy is getting cookies out of the cookie jar...,0
1,the boy's getting cookies out o the cookie ja...,0
2,oh goody . that boy's climbed on a stool that...,0
3,are you ready ? well the sink is overflowing ....,0
4,a boy and a girl are in the kitchen with their...,0


In [0]:
batch_1 = ad_df.append(nc_df, ignore_index=True)

We can ask pandas how many sentences are labeled as "positive" (value 1) and how many are labeled "negative" (having the value 0)

In [30]:
batch_1['label'].value_counts()

1    30
0    30
Name: label, dtype: int64

## Loading the Pre-trained BERT model
Let's now load a pre-trained BERT model. 

In [31]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Right now, the variable `model` holds a pretrained distilBERT model -- a version of BERT that is smaller, but much faster and requiring a lot less memory.

## Model #1: Preparing the Dataset
Before we can hand our sentences to BERT, we need to so some minimal processing to put them in the format it requires.

### Tokenization
Our first step is to tokenize the sentences -- break them up into word and subwords in the format BERT is comfortable with.

In [62]:
tokenized = batch_1['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
tokenized.head()

0    [101, 2821, 2879, 1012, 10303, 1012, 2155, 200...
1    [101, 2054, 1005, 1055, 6230, 2045, 1029, 2821...
2    [101, 2821, 2210, 2879, 1005, 1055, 1999, 1996...
3    [101, 2045, 1005, 1055, 1037, 2210, 2611, 4285...
4    [101, 2182, 1005, 1055, 1037, 17387, 15723, 10...
Name: text, dtype: object

### Padding
After tokenization, `tokenized` is a list of sentences -- each sentences is represented as a list of tokens. We want BERT to process our examples all at once (as one batch). It's just faster that way. For that reason, we need to pad all lists to the same size, so we can represent the input as one 2-d array, rather than a list of lists (of different lengths).

In [64]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
padded[0]

array([  101,  2821,  2879,  1012, 10303,  1012,  2155,  2003,  1999,
        1996,  3829,  1012,  1996,  2388,  1005,  1055, 12699, 10447,
        1012,  1998,  2014,  7752,  2003,  2058, 12314,  2075,  1012,
        1998,  2016,  1005,  1055,  2559,  2041,  1996,  3332,  1012,
        1998,  1996,  2048,  4268,  2024,  2635,  2027,  1005,  2128,
       11065, 16324,  2125,  1996,  2041,  1035,  1997,  1996, 25337,
        1012,  1998,  1996,  2879,  3504,  2066,  2002,  1005,  1055,
        6069,  2991,  2091,  1998,  3480,  2370,  2030,  2991,  2114,
        2010,  2388,  1012,  1998,  1996,  2611,  2003, 13550,  1523,
        2123,  1005,  1056,  2191,  2205,  2172,  5005,  1524,  2000,
        2032,  1012,  2016,  1005,  1055,  1004,  2292,  2030,  2842,
        2016,  1005,  1055,  4756,  2378,  2012,  2032,  1012,  2027,
        2288,  1996, 16324,  1012, 10303,  2085,  2295,  1996,  3332,
        1010,  2292,  1005,  1055,  2156,  1012,  2045,  1005,  1055,
        1037,  3835,

Our dataset is now in the `padded` variable, we can view its dimensions below:

In [35]:
np.array(padded).shape

(60, 357)

### Masking
If we directly send `padded` to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [65]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask[0]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [39]:
attention_mask.shape

(60, 357)

## Model #1: And Now, Deep Learning!
Now that we have our model and inputs ready, let's run our model!

The `model()` function runs our sentences through BERT. The results of the processing will be returned into `last_hidden_states`.

In [0]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)



In [49]:
last_hidden_states[0].size()

torch.Size([60, 357, 768])

Let's slice only the part of the output that we need. That is the output corresponding the first token of each sentence. The way BERT does sentence classification, is that it adds a token called `[CLS]` (for classification) at the beginning of every sentence. The output corresponding to that token can be thought of as an embedding for the entire sentence.


We'll save those in the `features` variable, as they'll serve as the features to our logitics regression model.

In [0]:
features = last_hidden_states[0][:,0,:].numpy()

The labels indicating which sentence is positive and negative now go into the `labels` variable

In [0]:
labels = batch_1['label']

## Model #2: Train/Test Split
Let's now split our datset into a training set and testing set.

In [0]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)


### [Bonus] Grid Search for Parameters
We can dive into Logistic regression directly with the Scikit Learn default parameters, but sometimes it's worth searching for the best value of the C parameter, which determines regularization strength.

In [0]:
# parameters = {'C': np.linspace(0.0001, 100, 20)}
# grid_search = GridSearchCV(LogisticRegression(), parameters)
# grid_search.fit(train_features, train_labels)

# print('best parameters: ', grid_search.best_params_)
# print('best scrores: ', grid_search.best_score_)

We now train the LogisticRegression model. If you've chosen to do the gridsearch, you can plug the value of C into the model declaration (e.g. `LogisticRegression(C=5.2)`).

In [54]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


## Evaluating Model #2
So how well does our model do in classifying sentences? One way is to check the accuracy against the testing dataset:

In [55]:
lr_clf.score(test_features, test_labels)

0.7333333333333333

How good is this score? What can we compare it against? Let's first look at a dummy classifier:

In [56]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.489 (+/- 0.36)


So our model clearly does better than a dummy classifier. But how does it compare against the best models? We need to go to the relevant literature for that.